In [1]:
import os
import torch
from utils import get_vocabs_dep_parser

In [2]:
data_dir = 'data_new'
path_train = os.path.join(data_dir, 'train.labeled')
path_test = os.path.join(data_dir, 'test.labeled')
# path_train = os.path.join(data_dir, 'train_short.labeled')
# path_test = os.path.join(data_dir, 'test_short.labeled')

# get only train vocabs to know which words are unknown in test
paths_list_train = [path_train]
word_dict_train, pos_dict_train = get_vocabs_dep_parser(paths_list_train)

paths_list_all = [path_train, path_test]
word_dict_all, pos_dict_all = get_vocabs_dep_parser(paths_list_all)

In [3]:
from data_handling import DepDataset
from torch.utils.data.dataloader import DataLoader
print("path_train -", path_train)
print("path_test -", path_test)
BATCH_SIZE = 1

train = DepDataset(word_dict_all, pos_dict_all, data_dir, 'train', padding=False)
train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=False)
test = DepDataset(word_dict_all, pos_dict_all, data_dir, 'test', padding=False, train_word_dict=word_dict_train)
test_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

# train = DepDataset(word_dict, pos_dict, data_dir, 'train_short', padding=False)
# train_dataloader = DataLoader(train, batch_size=BATCH_SIZE, shuffle=True)
# test = DepDataset(word_dict, pos_dict, data_dir, 'test_short', padding=False)
# test_dataloader = DataLoader(test, batch_size=BATCH_SIZE, shuffle=False)

path_train - data_new\train.labeled
path_test - data_new\test.labeled
idx_pos_mappings - [0, 1, 2, '#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']
pos_idx_mappings - {'<pad>': 0, '<unk>': 1, '<root>': 2, '#': 3, '$': 4, "''": 5, '(': 6, ')': 7, ',': 8, '.': 9, ':': 10, 'CC': 11, 'CD': 12, 'DT': 13, 'EX': 14, 'FW': 15, 'IN': 16, 'JJ': 17, 'JJR': 18, 'JJS': 19, 'LS': 20, 'MD': 21, 'NN': 22, 'NNP': 23, 'NNPS': 24, 'NNS': 25, 'PDT': 26, 'POS': 27, 'PRP': 28, 'PRP$': 29, 'RB': 30, 'RBR': 31, 'RBS': 32, 'RP': 33, 'SYM': 34, 'TO': 35, 'UH': 36, 'VB': 37, 'VBD': 38, 'VBG': 39, 'VBN': 40, 'VBP': 41, 'VBZ': 42, 'WDT': 43, 'WP': 44, 'WP$': 45, 'WRB': 46, '``': 47}
idx_pos_mappings - [0, 1, 2, '#', '$', "''", '(', ')', ',', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', '

In [4]:
print("Number of Train Sentences ", len(train))
print("Number of Test Sentences ",len(test))

Number of Train Sentences  5000
Number of Test Sentences  1000


In [ ]:
import torch
from torch import nn
import torch.optim as optim
from model import DnnSepParser
from trainer import Trainer
from loss import NllLoss
from utils import IGNORE_IDX

WORD_EMBEDDING_DIM = 100
TAG_EMBEDDING_DIM = 25
NUM_LAYERS = [2, 3, 4, 5, 6]
word_vocab_size = len(train.word_idx_mappings)
tag_vocab_size = len(train.pos_idx_mappings)
max_sentence_len = max(train.max_seq_len, test.max_seq_len)

acumulate_grad_steps = 50
len_train = len(train)
len_test = len(test)

for num_layer in NUM_LAYERS:
    model = DnnSepParser(WORD_EMBEDDING_DIM, TAG_EMBEDDING_DIM, num_layer, word_vocab_size, tag_vocab_size, max_sentence_len)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda:0" if use_cuda else "cpu")

    if use_cuda:
        model.cuda()

    # TODO: use our function with ignore index
    # loss_function = nn.NLLLoss(ignore_index=IGNORE_IDX)
#     loss_function = nn.NLLLoss()
    loss_function = NllLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    acumulate_grad_steps = 50 # This is the actual batch_size, while we officially use batch_size=1
    trainer = Trainer(model, optimizer, loss_function, device)

#     torch.manual_seed(1)
    EPOCHS = 100

    trainer.train_dep_parser(EPOCHS, train_dataloader, test_dataloader, acumulate_grad_steps, len_train, len_test, early_stopping=5)

tensor([[-5.6818, -7.4735, -7.5403, -5.9936, -1.9239, -6.1666, -5.2465, -1.9967,
         -5.7537, -6.3410, -2.6516, -6.0935, -1.8355, -6.4731, -2.2647, -6.2521,
         -2.9862, -4.6957, -7.6173, -5.5102, -2.3712, -5.7957, -2.1291, -4.5326,
         -6.1532, -5.7552, -2.7543],
        [-5.2558, -6.8507, -6.9314, -5.4912, -1.9917, -5.6839, -4.8899, -2.0653,
         -5.3224, -5.7999, -2.5966, -5.5804, -1.8625, -5.9163, -2.3013, -5.7352,
         -3.0142, -4.4456, -7.0152, -5.1079, -2.3579, -5.3491, -2.2078, -4.3348,
         -5.6650, -5.3040, -2.6838],
        [-5.9484, -7.4718, -7.5311, -6.0968, -1.7503, -6.4754, -5.4487, -1.8939,
         -6.0902, -6.4771, -2.9172, -6.2186, -1.7504, -6.5557, -2.3128, -6.4044,
         -3.2906, -5.1238, -7.6109, -5.7305, -2.5263, -5.9196, -1.9863, -4.9368,
         -6.4009, -5.9646, -2.9926],
        [-3.8619, -4.8232, -4.9425, -4.3884, -2.4595, -3.9651, -4.0768, -2.4759,
         -3.8648, -4.3826, -2.6221, -4.3773, -2.3251, -4.4842, -2.5440, -4.2554